In [1]:
import pandas as pd
import csv
import re
import string
import nltk

from pythainlp.tokenize import Tokenizer
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import dict_trie
from pythainlp.tag import pos_tag

from nltk.corpus import words
from stop_words import get_stop_words

# Input

In [2]:
# input
# import data from csv
file_name = 'category1'

FILE = './Data/{}.csv'.format(file_name)
df = pd.read_csv(FILE)

In [3]:
df.head()

,Unnamed: 0,category_0_0,category_0_1,category_0_2,category_0_3,category_0_4,category_0_5,category_0_6,category_0_7,category_0_8,...,category_33_234,category_33_235,category_33_236,category_33_237,category_33_238,category_33_239,category_33_240,category_33_241,category_33_242,category_33_243
0,0,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,...,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New
1,1,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,...,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต
2,2,Samsung Galaxy M30s (4/64GB) ( โทรศัพท์มือถือ ),โทรศัพท์มือถือ Samsung Galaxy A50s Ram4/64GB (...,Samsung Galaxy A10s (RAM2GB/ROM32GB),Samsung Galaxy Note 10 + (12/256 GB) ( โทรศัพท...,Samsung Galaxy A7 (2017),Samsung Galaxy A9 (2018) 6 GB RAM 128 GB ROM 6...,Samsung A30S (RAM 4GB / ROM 64 GB) มือ 1 ประกั...,(มือ1 ศูนย์ไทย) Samsung Galaxy Note 10 Plus No...,Samsung Galaxy A70 (8/128GB) ( โทรศัพท์มือถือ ),...,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New
3,3,แบตอึดอยู่นาน สนุกได้ทั้งวัน กับความจุ 6000 mA...,GSM 850/900/1800/1900MHz,สมาร์ทโฟน (โทรศัพท์มือถือพร้อมระบบปฏิบัติการ),ขนาดหน้าจอ 6.8 นิ้ว,ระบบเซ็นเซอร์ (Sensor)\r\n- ระบบตรวจสอบลายนิ้ว...,จอแสดงผล Super AMOLED 24-bit (16 ล้านสี)Fullvi...,คุณสมบัติ Samsung A30S,Samsung Galaxy Note10 plus TH ประกันศูนย์ไทย 1...,Galaxy A70 (SM-A705FZBGTHL),...,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New
4,4,ระบบกล้องหลัง 3 ตัว ที่มีกล้องชัดสูงสุดถึง 48 ...,3G WCDMA 850/900/1900/2100MHz,จอแสดงผล TFT-LCD 24-bit (16 ล้านสี)- หน้าจอทรง...,Ram 12 GB,คุณสมบัติการกันน้ำ (Waterproof)\r\n- กันน้ำได้...,ระบบตรวจสอบลายนิ้วมือ (Fingerprint)ระบบจดจำใบห...,จอแสดงผล Super AMOLED 24-bit (16 ล้านสี),แบรนด์ ซัมซุง,สเปคเทพของเกมเมอร์,...,GloryStar BASEUS PUBG ไฟฟรีเกมที่วางโทรศัพท์แผ...,GloryStar นาฬิกาติดตามตำแหน่งอัจฉริยะเด็กนาฬิก...,GloryStar Joy-CON ตัวควบคุมเกมแป้นเกมส์จอยเกมส...,GloryStar คอนโซลเกมคลาสสิก 8 BitsProfessional ...,GloryStar สีน้ำเงินแปรงเกมควบคุมเครื่องรับสัญญ...,GloryStar Flysky รีโมทคอนโทรล FS-IT4S + IA4B 4...,Top-Sky PlayStation 2 PS2 Memory Card - 32MB,Top-Sky PlayStation 2 PS2 Memory Card - 256MB,Top-Sky PlayStation 2 PS2 Memory Card - 128MB,Top - Sky 3 - in - 1 PS4/PS4 Slim/PS4 เกมโปรที...


In [4]:
# Drop a column that not use
data = df.drop(columns=['Unnamed: 0'])

# Drop column that all equal to NaN
data = data.dropna(axis=1, how='all')

# Fill NaN with "-"
data = data.fillna('-')

In [5]:
data.head()

,category_0_0,category_0_1,category_0_2,category_0_3,category_0_4,category_0_5,category_0_6,category_0_7,category_0_8,category_0_9,...,category_33_234,category_33_235,category_33_236,category_33_237,category_33_238,category_33_239,category_33_240,category_33_241,category_33_242,category_33_243
0,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,...,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New
1,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,...,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต
2,Samsung Galaxy M30s (4/64GB) ( โทรศัพท์มือถือ ),โทรศัพท์มือถือ Samsung Galaxy A50s Ram4/64GB (...,Samsung Galaxy A10s (RAM2GB/ROM32GB),Samsung Galaxy Note 10 + (12/256 GB) ( โทรศัพท...,Samsung Galaxy A7 (2017),Samsung Galaxy A9 (2018) 6 GB RAM 128 GB ROM 6...,Samsung A30S (RAM 4GB / ROM 64 GB) มือ 1 ประกั...,(มือ1 ศูนย์ไทย) Samsung Galaxy Note 10 Plus No...,Samsung Galaxy A70 (8/128GB) ( โทรศัพท์มือถือ ),Smartphone โทรศัพท์มือถือ Samsung Galaxy A10S ...,...,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New
3,แบตอึดอยู่นาน สนุกได้ทั้งวัน กับความจุ 6000 mA...,GSM 850/900/1800/1900MHz,สมาร์ทโฟน (โทรศัพท์มือถือพร้อมระบบปฏิบัติการ),ขนาดหน้าจอ 6.8 นิ้ว,ระบบเซ็นเซอร์ (Sensor)\r\n- ระบบตรวจสอบลายนิ้ว...,จอแสดงผล Super AMOLED 24-bit (16 ล้านสี)Fullvi...,คุณสมบัติ Samsung A30S,Samsung Galaxy Note10 plus TH ประกันศูนย์ไทย 1...,Galaxy A70 (SM-A705FZBGTHL),แบรนด์ ซัมซุง,...,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New
4,ระบบกล้องหลัง 3 ตัว ที่มีกล้องชัดสูงสุดถึง 48 ...,3G WCDMA 850/900/1900/2100MHz,จอแสดงผล TFT-LCD 24-bit (16 ล้านสี)- หน้าจอทรง...,Ram 12 GB,คุณสมบัติการกันน้ำ (Waterproof)\r\n- กันน้ำได้...,ระบบตรวจสอบลายนิ้วมือ (Fingerprint)ระบบจดจำใบห...,จอแสดงผล Super AMOLED 24-bit (16 ล้านสี),แบรนด์ ซัมซุง,สเปคเทพของเกมเมอร์,SKU 571250946_TH-1048780375,...,GloryStar BASEUS PUBG ไฟฟรีเกมที่วางโทรศัพท์แผ...,GloryStar นาฬิกาติดตามตำแหน่งอัจฉริยะเด็กนาฬิก...,GloryStar Joy-CON ตัวควบคุมเกมแป้นเกมส์จอยเกมส...,GloryStar คอนโซลเกมคลาสสิก 8 BitsProfessional ...,GloryStar สีน้ำเงินแปรงเกมควบคุมเครื่องรับสัญญ...,GloryStar Flysky รีโมทคอนโทรล FS-IT4S + IA4B 4...,Top-Sky PlayStation 2 PS2 Memory Card - 32MB,Top-Sky PlayStation 2 PS2 Memory Card - 256MB,Top-Sky PlayStation 2 PS2 Memory Card - 128MB,Top - Sky 3 - in - 1 PS4/PS4 Slim/PS4 เกมโปรที...


In [6]:
# Filter
data = data.filter(regex ='category_[0-9]_[0-9]+')
#data_01.head(20)

# Clean text

In [7]:
file_name2 = './Data/attributes_category1.txt'
with open(file_name2, 'r', encoding='utf-8') as f:
    attr_fix = f.read().splitlines()

In [8]:
regex_attr = ("|").join(attr_fix)

In [9]:
# clean the text and prepre some attribute
def clean_text_category1(msg):
    msg = str(msg).lower()
    msg = re.sub(r"smartphone[s]?","สมาร์ทโฟน",str(msg))
    msg = re.sub(r"computers และ laptops","คอมพิวเตอร์ และ แล็ปท็อป",str(msg))
    msg = re.sub(r"laptop[s]?","แล็ปท็อป",str(msg))
    msg = re.sub(r"desktops computers","คอมพิวเตอร์แบบตั้งโต๊ะ",str(msg))    
    
    if str(msg)=='โทรศัพท์มือถือและแท็บเล็ต':
        msg = re.sub(r"(โทรศัพท์มือถือและแท็บเล็ต)",r"หมวดหมู่ \1",str(msg))
    elif (str(msg)=='สมาร์ทโฟน') | (str(msg)=='แท็บเล็ต'):
        msg = re.sub(r"(สมาร์ทโฟน|แท็บเล็ต)",r"ประเภทย่อย \1",str(msg))
    elif str(msg)=='คอมพิวเตอร์ และ แล็ปท็อป':
        msg = re.sub(r"(คอมพิวเตอร์ และ แล็ปท็อป)",r"หมวดหมู่ \1",str(msg))
    elif (str(msg)=='แล็ปท็อป') | (str(msg)=='แล็ปท็อปสำหรับเล่นเกมส์'):
        msg = re.sub(r"(แล็ปท็อป|แล็ปท็อปสำหรับเล่นเกมส์)",r"ประเภทย่อย \1",str(msg))
    elif (str(msg)=='คอมพิวเตอร์แบบตั้งโต๊ะ') | (str(msg)=='gaming desktops') | (str(msg)=='คอมประกอบ'):
        msg = re.sub(r"(คอมประกอบ|คอมพิวเตอร์แบบตั้งโต๊ะ|gaming desktops)",r"ประเภทย่อย \1",str(msg))
    elif re.match("[0-9]\*[0-9]",str(msg)):
        msg = re.sub(r"\*","x",str(msg))
    
    msg = re.sub(r"\&","และ",str(msg))
    msg = re.sub(r"sku","sku ",str(msg))
    msg = re.sub(r"\%","เปอร์เซนต์ ",str(msg))
    msg = re.sub(r"brand","แบรนด์ ",str(msg))
    msg = re.sub(r"model","โมเดล ",str(msg))
    msg = re.sub(r"กราฟฟิก|กราฟิก","graphics",str(msg))
    msg = re.sub(r"^color|^colour","สี ",str(msg))
    msg = re.sub(r"พาวเวอร์ซัพพลาย\s|power\ssupply\s","psu ",str(msg))
    msg = re.sub(r"เมนบอร์ด\s|^mb\s","mainboard ",str(msg))
    msg = re.sub(r"warranty\speriod\s+?","ระยะเวลาการรับประกัน ",str(msg))
    msg = re.sub(r"\sprocessor\s","cpu ",str(msg))
    msg = re.sub(r"เลโนโว่"," lenovo ",str(msg))
    msg = re.sub(r"ซัมซุง"," samsung ",str(msg))
    
    msg = re.sub(r"จอแสดงผล\s(size|screen)|[\W]จอแสดงผล\s|^screen\s|^monitor\s","หน้าจอแสดงผล ",str(msg))
    msg = re.sub(r"^resolution\s+?","ความละเอียด ",str(msg))
    msg = re.sub(r"^os\:?\s|operating\system\s+?","ระบบปฏิบัติการ ",str(msg))
    msg = re.sub(r"ความจุ\s*\(storage\)","ความจุที่เก็บข้อมูล ",str(msg))
    msg = re.sub(r"การ์ดจอ|การ์ดแสดงผล|graphics|graphic[s]? card|vga","gpu ",str(msg))
    msg = re.sub(r"ซีพียู\s","cpu ",str(msg))
    msg = re.sub(r"[\d+\s+]mp|เมกะพิกเซล\s?","ล้านพิกเซล ",str(msg))
    msg = re.sub(r"\s+แรม\s?|main\smemory\s|^memory\s+?[^r]"," ram ",str(msg))
    msg = re.sub(r"\s+รอม\s?"," rom ",str(msg))
    msg = re.sub(r"ความจุฮาร์ดไดร์ฟ|ฮาร์ดดิสก์\s?|hard\s?(disk|drive)\s?|hdd size"," hdd ",str(msg))
    msg = re.sub(r"display","จอแสดงผล ",str(msg))
    msg = re.sub(r"weight?","น้ำหนัก ",str(msg))
    msg = re.sub(r"[b]attery\s?[c]apacity","ความจุแบตเตอรี่ ",str(msg))
    msg = re.sub(r"แบต\s?^เตอรี่","แบตเตอรี่ ",str(msg))
    msg = re.sub(r"\w*\s?dimension w x d x h","ขนาด ",str(msg))
    msg = re.sub(r"\w*\s?dimension","ขนาด ",str(msg))
    msg = re.sub(r"พอร์ต|ports\s","port ",str(msg))
    
    # หน่วย
    msg = re.sub(r"\฿,"," บาท ",str(msg))
    msg = re.sub(r"\sมิลลิเมตร|\sมม\.?|millimeter\.?"," mm ",str(msg))
    msg = re.sub(r"\sเซนติเมตร|\sซม\.?|centimeter\.?"," cm ",str(msg))
    msg = re.sub(r"\sกิโลกรัม|\sกก\.?|kilogram[s]?\.?"," kg ",str(msg))
    msg = re.sub(r"\sนิ้ว"," inch ",str(msg))
    
    msg = re.sub(r"^(amd ryzen|intel\w*[ip])",r"cpu \1",str(msg))
    msg = re.sub(r"^(nvidia|intel\w*graphic)",r"gpu \1",str(msg))
    msg = re.sub(r"^(\d+\.?\d?\sinch)",r"ขนาดหน้าจอ \1",str(msg))
    msg = re.sub(r"^(windows)",r"ระบบปฏิบัติการ \1",str(msg))
    msg = re.sub(r"\s(\d+\s)month[s]?",r"\1 เดือน",str(msg))
    msg = re.sub(r"\s(\d+\s)year[s]?",r"\1 ปี",str(msg))
    
    
    # clean special character
    msg = re.sub(r"[\,]","",str(msg))
    msg = re.sub(r"[^a-z0-9ก-๙\.\s]"," ", str(msg))
    msg = re.sub(r"\s+"," ",str(msg)) 
    
    if any(ext in msg for ext in attr_fix):
        return msg
    else:
        return

In [10]:
# clean name
def clean_name(msg):
    msg = str(msg).lower()
    
    msg = re.sub(r"\&","และ",str(msg))
    msg = re.sub(r"sku","sku ",str(msg))
    msg = re.sub(r"brand","แบรนด์ ",str(msg))
    msg = re.sub(r"model","โมเดล ",str(msg))
    msg = re.sub(r"[\d\s]*มิลลิเมตร|\sมม\.?|millimeter\.?"," mm ",str(msg))
    msg = re.sub(r"[\d\s]*เซนติเมตร|\sซม\.?|centimeter\.?"," cm ",str(msg))
    msg = re.sub(r"\s+"," ",str(msg))
    
    return msg
    

In [11]:
# function for defining attribute of product
def define_attr(txt):
    p = re.compile('(หมวดหมู่|ประเภทย่อย|{}|)'.format(regex_attr), re.VERBOSE)
    #match = re.match(r"", str(txt))
    txt = re.sub(p,r"|\1|",str(txt))
    
    return txt

In [12]:
# Cleaning text with clean_text function and keep in dict
data_clean = {}
name_item = {}
for key,value in data.iteritems():
    tmp = []
    for i,txt in enumerate(value):
        if not txt == '-':
            if i==2 and re.match(r"category_[01]_\d+",str(key)):
                name_item[key] = clean_name(txt)
            elif i==3 and re.match(r"category_[23456789]_\d+",str(key)):
                name_item[key] = clean_name(txt)
            else:
                tmp.append(clean_text_category1(txt))
    
    tmp = [i for i in tmp if i] # Clear None value
    data_clean[key] = tmp 


In [13]:
# Show some of data in dict
name_item

{'category_0_0': 'samsung galaxy m30s (4/64gb) ( โทรศัพท์มือถือ )',
 'category_0_1': 'โทรศัพท์มือถือ samsung galaxy a50s ram4/64gb (รับประกันศูนย์)by jaymart',
 'category_0_2': 'samsung galaxy a10s (ram2gb/rom32gb)',
 'category_0_3': 'samsung galaxy note 10 + (12/256 gb) ( โทรศัพท์มือถือ )',
 'category_0_4': 'samsung galaxy a7 (2017)',
 'category_0_5': 'samsung galaxy a9 (2018) 6 gb ram 128 gb rom 6.3 นิ้ว 5 กล้อง สูงสุด 24 mp tetra cell (f1.7) ( โทรศัพท์มือถือ )',
 'category_0_6': 'samsung a30s (ram 4gb / rom 64 gb) มือ 1 ประกันศูนย์ 1 ปี',
 'category_0_7': '(มือ1 ศูนย์ไทย) samsung galaxy note 10 plus note 10 + (12/256gb) ประกัน1ปี',
 'category_0_8': 'samsung galaxy a70 (8/128gb) ( โทรศัพท์มือถือ )',
 'category_0_9': 'smartphone โทรศัพท์มือถือ samsung galaxy a10s sm-a107m ขนาดหน้าจอ 6.2 นิ้ว ram 2 / rom 32 gb (สีน้ำเงิน)',
 'category_0_10': 'samsung galaxy a5 (2017)',
 'category_0_11': 'samsung galaxy a5 (2017) - gold',
 'category_0_12': 'samsung galaxy s10e (6/128gb) ( โทรศัพท์มือถือ

In [14]:
# Mark attribute with '|'
attr_mark = {}
for key,value in data_clean.items():
    tmp = [define_attr(txt) for txt in value] 
    tmp = [re.sub(r"\|\|","",str(t)) for t in tmp]
    tmp = [i for i in tmp if re.search(r"\|",i)] 
    attr_mark[key] = tmp 

In [15]:
attr_mark

{'category_0_0': ['|หมวดหมู่| โทรศัพท์มือถือและแท็บเล็ต',
  '|ประเภทย่อย| สมาร์ทโฟน',
  'แบตอึดอยู่นาน สนุกได้ทั้งวัน กับ|ความจุ| 6000 mah และสามารถชาร์จแบตเร็ว ด้วย fast charging',
  'ระบบ|กล้องหลัง| 3 ตัว ที่มีกล้องชัดสูงสุดถึง 48ล้านพิกเซล มาพร้อมกับ|เลนส์|มุมกว้างพิเศษ ultra wide',
  'ใช้ exynos 9611 สเป็กแรงขึ้นเล่นเกมส์ได้ไม่มีสะดุด กับ |ram| 4 gb |rom| 64 gb',
  '|หน้าจอ|แสดงผล super amoled infinity u |ขนาด| 6.4 inch ',
  '|แบรนด์| samsung ',
  '|sku| 480942616 th 870640886',
  '|โมเดล| galaxy m30s 4 64gb ',
  '|ระยะเวลาการรับประกัน| 1 ปี'],
 'category_0_1': ['|หมวดหมู่| โทรศัพท์มือถือและแท็บเล็ต',
  '|ประเภทย่อย| สมาร์ทโฟน',
  '|รองรับ| 2 ซิม nano sim',
  '|หน่วยประมวลผล| exynos 9610 quad 2.3 ghz quad 1.7 ghz',
  '|ระบบปฏิบัติการ| android',
  'หน่วยความจำภายใน 64gb เพิ่ม micro sd |รองรับ|สูงสุด 512 gb',
  '|ram| 4 gb',
  '|หน้าจอ| 6.4 inch |ความละเอียด| 1080 x 2340 พิกเซล fhd ',
  '|กล้องหลัง| 3 |เลนส์| 48 ล้านพิกเซล f 2.0 5 ล้านพิกเซล f 2.2 bokeh 8 ล้านพิกเซล f 2.4 ultrawide แ

# Capture attribute manual

In [16]:
# Split attribute
def cap_attr(text):
    text = text.split("|")
    
    return text

In [17]:
# capture attribute by split the marker
attr = {}
for i,v in attr_mark.items():
    tmp_list = []
    for txt in v:
        match = cap_attr(txt)
        match = [m for m in match if not m =='' and not m == ' ']
        tmp_list.append(match)
    attr[i] = tmp_list

In [18]:
attr

{'category_0_0': [['หมวดหมู่', ' โทรศัพท์มือถือและแท็บเล็ต'],
  ['ประเภทย่อย', ' สมาร์ทโฟน'],
  ['แบตอึดอยู่นาน สนุกได้ทั้งวัน กับ',
   'ความจุ',
   ' 6000 mah และสามารถชาร์จแบตเร็ว ด้วย fast charging'],
  ['ระบบ',
   'กล้องหลัง',
   ' 3 ตัว ที่มีกล้องชัดสูงสุดถึง 48ล้านพิกเซล มาพร้อมกับ',
   'เลนส์',
   'มุมกว้างพิเศษ ultra wide'],
  ['ใช้ exynos 9611 สเป็กแรงขึ้นเล่นเกมส์ได้ไม่มีสะดุด กับ ',
   'ram',
   ' 4 gb ',
   'rom',
   ' 64 gb'],
  ['หน้าจอ', 'แสดงผล super amoled infinity u ', 'ขนาด', ' 6.4 inch '],
  ['แบรนด์', ' samsung '],
  ['sku', ' 480942616 th 870640886'],
  ['โมเดล', ' galaxy m30s 4 64gb '],
  ['ระยะเวลาการรับประกัน', ' 1 ปี']],
 'category_0_1': [['หมวดหมู่', ' โทรศัพท์มือถือและแท็บเล็ต'],
  ['ประเภทย่อย', ' สมาร์ทโฟน'],
  ['รองรับ', ' 2 ซิม nano sim'],
  ['หน่วยประมวลผล', ' exynos 9610 quad 2.3 ghz quad 1.7 ghz'],
  ['ระบบปฏิบัติการ', ' android'],
  ['หน่วยความจำภายใน 64gb เพิ่ม micro sd ', 'รองรับ', 'สูงสุด 512 gb'],
  ['ram', ' 4 gb'],
  ['หน้าจอ', ' 6.4 inch ', 'ค

In [19]:
# Construct triples of product
triples = []
for i,v in attr.items():
    for l in v:
        idx=0
        for idx in range(len(l)-1):
            tmp_list = [i]
            if l[idx] in attr_fix:
                tmp_list.extend([l[idx],l[idx+1]])
                triples.append(tmp_list)
            idx += 1     

In [20]:
triples

[['category_0_0', 'หมวดหมู่', ' โทรศัพท์มือถือและแท็บเล็ต'],
 ['category_0_0', 'ประเภทย่อย', ' สมาร์ทโฟน'],
 ['category_0_0',
  'ความจุ',
  ' 6000 mah และสามารถชาร์จแบตเร็ว ด้วย fast charging'],
 ['category_0_0',
  'กล้องหลัง',
  ' 3 ตัว ที่มีกล้องชัดสูงสุดถึง 48ล้านพิกเซล มาพร้อมกับ'],
 ['category_0_0', 'เลนส์', 'มุมกว้างพิเศษ ultra wide'],
 ['category_0_0', 'ram', ' 4 gb '],
 ['category_0_0', 'rom', ' 64 gb'],
 ['category_0_0', 'หน้าจอ', 'แสดงผล super amoled infinity u '],
 ['category_0_0', 'ขนาด', ' 6.4 inch '],
 ['category_0_0', 'แบรนด์', ' samsung '],
 ['category_0_0', 'sku', ' 480942616 th 870640886'],
 ['category_0_0', 'โมเดล', ' galaxy m30s 4 64gb '],
 ['category_0_0', 'ระยะเวลาการรับประกัน', ' 1 ปี'],
 ['category_0_1', 'หมวดหมู่', ' โทรศัพท์มือถือและแท็บเล็ต'],
 ['category_0_1', 'ประเภทย่อย', ' สมาร์ทโฟน'],
 ['category_0_1', 'รองรับ', ' 2 ซิม nano sim'],
 ['category_0_1', 'หน่วยประมวลผล', ' exynos 9610 quad 2.3 ghz quad 1.7 ghz'],
 ['category_0_1', 'ระบบปฏิบัติการ', ' android'

# 2 outputs

In [21]:
# Write the triples to text file
with open('./Data/triples_{}.txt'.format(file_name), 'w', encoding='utf-8') as f:
    for _list in triples:
        for _string in _list:
            #f.seek(0)
            f.write(str(_string) + '\t')
        f.write('\n')

In [22]:
# Product name and index
with open('./Data/Product_names_{}.txt'.format(file_name), 'w', encoding='utf-8') as fn:
    print(name_item, file=fn)

# Proprocessing text

# download stopwords of TH and EN
nltk.download('words')
th_stop = tuple(thai_stopwords())
en_stop = tuple(get_stop_words('en'))

# path of custom dictionary for word tokenize
PATH_TO_CUSTOM_DICT = './Data/custom_dict.txt'

from pythainlp.tokenize import sent_tokenize

# split text to words 
def split_word(text):
    
    # Split text to words
    _tokenizer = Tokenizer(custom_dict=PATH_TO_CUSTOM_DICT, engine='newmm')
    words = sent_tokenize(text)

    # Remove stop words in TH and EN
    words = [i for i in words if not i in th_stop and not i in en_stop]
    
    # Delete whitespace
    words = [i for i in words if not ' ' in i]
                

    return words

## Tokenize text

token = {}
for key,value in data_clean.items():
    tmp = [split_word(txt) for txt in value]
    token[key] = tmp

token_list = {}
for i,v in token.items():
    token_list[i] = [item for sublist in v for item in sublist]

attr_list = {}
res = []
for i,v in attr.items():
    tmp_list = []
    for t in v:
        if type(t) != list:
            t_split = t.split(':')
            if len(t_split)<3:
                tmp_list.append(t_split)
        else:
            for ele in t:
                if (ele[0]!='' and ele[1]!=''):
                    res = [list(ele)]
            
            for l in res:
                tmp_list.append(l)
    attr_list[i] = tmp_list

attr_list

dict_attr = {}
dict_value = {}
for i,v in attr_list.items():
    tmp_attr = []
    tmp_value = []
    for e in v:
        tmp_attr.append(e[0])
        tmp_value.append(e[1])
    dict_attr[i] = tmp_attr
    dict_value[i] = tmp_value

dict_value

series = { key: pd.Series(val) for key, val in token.items() }

token2 = {}
for i,v in series.items():
    token2[i] = v.apply(pd.Series).stack()

pd.DataFrame.from_dict(token2, orient='index').transpose()

## Category 1

token2

# Save data to csv file

# Transform dict to dataframe and save to csv file
pd.DataFrame.from_dict(data_clean, orient='index').transpose().reset_index().to_csv("./Data/{}_clean.csv".format(file_name))

pd.DataFrame.from_dict(data_clean, orient='index').transpose().reset_index().to_excel('./Data/category1_clean.xlsx',engine='xlsxwriter')

pd.DataFrame.from_dict(dict_value, orient='index').transpose().to_csv("./Data/{}_value.csv".format(file_name))